# Libraries

In [1]:
import time
import os
import json
import logging
from pathlib import Path

# Config

In [2]:
# === Base units ===
kN = 1.0
N = kN / 1000
cm = 1.0
m = cm * 100

# === Derived geometry ===
cm2 = cm**2
m2 = m**2
cm3 = cm**3
m3 = m**3

# === Line load ===
kN_per_m = kN / m

# === Pressure ===
Pa  = N / m2
kPa = 1e3 * Pa
MPa = 1e6 * Pa
GPa = 1e9 * Pa

# === Volumetric (density, unit weight, etc.) ===
kN_per_m3 = kN / m3

In [3]:
10 * MPa

0.9999999999999999

# Main parameters

In [4]:
# from plot_results import flat_data

# with open("scenarios_results.json") as f:
#     data = json.load(f)

# df = flat_data(data).drop([], axis=1)
# colu = df.drop(
#     [c for c in df.columns if c.endswith("Key")]
#     + [c for c in df.columns if c.startswith("Deleted_layer")]
#     + [c for c in df.columns if c.startswith(("Ux", "Uy", "Uz", "R1", "R2", "R3"))]
#     + ["Analysis", "_source_index"],
#     axis=1,
# )

# colu.describe().iloc[:, :20]


In [5]:
param_ranges = {}

# Parameter ranges (min, max)
param_ranges.update({
    "Foundation_w": (10 * kN_per_m3, 40 * kN_per_m3),
    "Foundation_Ehor": (1 * GPa, 30 * GPa),

    "Foundation_FtmHor": (0.01 * MPa, 4 * MPa),
    "Foundation_Gt": (0.001 * kN_per_m, 0.1 * kN_per_m),
    
    "Foundation_FmHor": (1 * MPa, 40 * MPa),
    "Foundation_Gc": (0.1 * kN_per_m, 10 * kN_per_m),

    "Foundation_Gd": (0.5 * GPa, 15 * GPa),

    "Foundation_fvk0d": (0.001 * MPa, 1 * MPa),
    "Foundation_FrictionRatioShear": (0.2, 1),
    "Foundation_ShearMaxTensileRatio": (0.3, 0.8),
    
    "Foundation_CohesionSlidingHor": (0.001 * MPa, 1 * MPa),
    "Foundation_FrictionRatioSlidingHor": (0.1, 1.2),
})

param_ranges.update({
    "Masonry_w": (10 * kN_per_m3, 40 * kN_per_m3),
    "Masonry_Ehor": (1 * GPa, 30 * GPa),

    "Masonry_FtmHor": (0.01 * MPa, 4 * MPa),
    "Masonry_Gt": (0.001 * kN_per_m, 0.1 * kN_per_m),
    
    "Masonry_FmHor": (1 * MPa, 40 * MPa),
    "Masonry_Gc": (0.1 * kN_per_m, 10 * kN_per_m),

    "Masonry_Gd": (0.5 * GPa, 15 * GPa),

    "Masonry_fvk0d": (0.001 * MPa, 1 * MPa),
    "Masonry_FrictionRatioShear": (0.2, 1),
    "Masonry_ShearMaxTensileRatio": (0.3, 0.8),
    
    "Masonry_CohesionSlidingHor": (0.001 * MPa, 1 * MPa),
    "Masonry_FrictionRatioSlidingHor": (0.1, 1.2),
})

param_ranges.update({
    "Backfill_w": (10 * kN_per_m3, 30 * kN_per_m3),
    "Backfill_Ehor": (0.001 * GPa, 4 * GPa),

    "Backfill_FtmHor": (0.0001 * kPa, 1000 * kPa),
    "Backfill_Gt": (0.001 * kN_per_m, 0.1 * kN_per_m),
    
    "Backfill_FmHor": (0.1 * MPa, 10 * MPa),
    "Backfill_Gc": (0.1 * kN_per_m, 10 * kN_per_m),

    "Backfill_Gd": (0.0005 * GPa, 2 * GPa),

    "Backfill_fvk0d": (0.0001 * kPa, 1000 * kPa),
    "Backfill_FrictionRatioShear": (0.2, 1),
    "Backfill_ShearMaxTensileRatio": (0.3, 0.8),
    
    "Backfill_CohesionSlidingHor": (0.0001 * kPa, 1000 * kPa),
    "Backfill_FrictionRatioSlidingHor": (0.1, 1.2),
})

param_ranges["Foundation_Soil_Ehor"] = (.01 * kPa, 0.5 * MPa)

param_ranges["Damaged_Ehor"] = (.01 * kPa, 0.5 * MPa)

param_ranges = {
    "Damaged_Ehor": (0.1 * MPa, 0.101 * MPa)
}

In [6]:
correlations = {
    # ("Masonry_Ehor", "Masonry_FmHor"): .5,
    # ("Masonry_Ehor", "Masonry_FtmHor"): 0.5,
    # ("Foundation_Ehor", "Foundation_FmHor"): 0.6,
    # ("Backfill_Gd", "Backfill_Ehor"): 0.4,
}

In [7]:
from build_scenarios import build_scenarios



# param_ranges = {}
n_scenarios = 4

scenarios = build_scenarios(param_ranges, n_scenarios, "NewAnalysis", correlations)

In [8]:
import random

locations = ["pier_1", "pier_2"]
materials = ["Damaged", "Foundation_Soil"]

for s in scenarios:
    s["Scour"] = {
        loc: random.choice(materials)
        for loc in locations
    }

In [9]:
scenarios[0]['Scour'] = {
    "pier_1": "Foundation_Soil",
    "pier_2": "Foundation_Soil"
}

scenarios[1]['Scour'] = {
    "pier_1": "Damaged",
    "pier_2": "Foundation_Soil"
}

scenarios[2]['Scour'] = {
    "pier_1": "Foundation_Soil",
    "pier_2": "Damaged"
}

scenarios[3]['Scour'] = {
    "pier_1": "Damaged",
    "pier_2": "Damaged"
}

# Run

In [10]:
from processing_steps import prepare_model
from run_scenario import run_scenario

from concurrent.futures import ThreadPoolExecutor, as_completed
def run_model(input_path, scenarios, timeout=360, max_workers=4):
    """Run all scenarios in parallel."""
    prepare_model(input_path)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for i, scenario in enumerate(scenarios):
            time.sleep(0.2)
            futures.append(executor.submit(run_scenario, input_path, scenario, i, timeout))
        for future in as_completed(futures):
            try:
                future.result()
            except KeyboardInterrupt:
                print("\n🛑 Program interrupted by user. Cancelling all remaining tasks...")
                executor.shutdown(wait=False, cancel_futures=True)
                raise
            except Exception as e:
                print(f"Error in one of the scenarios: {e}")
                time.sleep(0.2)


In [ ]:
import tempfile
import shutil

INPUT_FILE = r"Bridge_2_foundation.hrx"
INPUT_FILE = os.path.abspath(INPUT_FILE)

directory = os.path.dirname(INPUT_FILE)

# with tempfile.TemporaryDirectory(prefix="histra_run_", dir=directory) as temp_dir:
#     input_copy = os.path.join(temp_dir, os.path.basename(INPUT_FILE))
#     shutil.copy2(INPUT_FILE, input_copy)
#     run_model(input_copy, scenarios, timeout=360, max_workers=3)


temp_dir = os.path.join(directory, "temp_dir")
os.makedirs(temp_dir, exist_ok=True)
input_copy = os.path.join(temp_dir, os.path.basename(INPUT_FILE))
shutil.copy2(INPUT_FILE, input_copy)
run_model(input_copy, scenarios, timeout=360, max_workers=3)

Creating mesh analysis
Running mesh analysis
Pre-processing: 1
Copying: 1
Pre-processing: 2
Copying: 2
Pre-processing: 3
Copying: 3
Updating materials: 1
Updating materials: 2
Updating materials: 3
Updating foundation interfaces: 1
Updating foundation interfaces: 3
Updating foundation interfaces: 2
Pre-processing finished: 3
Pre-processing finished: 2
Pre-processing finished: 1
Processing: 3
Processing: 1Processing: 2

Running: 1 - NewAnalysis
Running: 3 - NewAnalysis
Running: 2 - NewAnalysis
Pos-processing: 3
Pre-processing finished: 3
Pre-processing: 4
Copying: 4
Updating materials: 4
Updating foundation interfaces: 4
Pre-processing finished: 4
Processing: 4
Running: 4 - NewAnalysis
⏰ Timeout: c:\Users\mbonatte\Documents\Coding\histra-automation\temp_dir\Bridge_2_foundation_copy_1.hrx exceeded 180 seconds. Killing...
Error in one of the scenarios: Command '['C:\\Program Files\\Gruppo Sismica\\HiStrA Bridges 2024.1.1\\SolverHistra.exe', 'run', 'c:\\Users\\mbonatte\\Documents\\Coding\\

In [12]:
scenarios[0]['Outputs']

KeyError: 'Outputs'

In [ ]:
a

# Save

In [ ]:
# Load existing data
try:
    with open("scenarios_results.json", "r") as f:
        data = json.load(f)
except FileNotFoundError:
    data = [] 

print(len(data))
# Append the new data
data.extend(scenarios)
print(len(data))

In [ ]:
from plot_results import flat_data
from filter_data import normalized_distance_filter

df = flat_data(data)

df = df.drop_duplicates(subset="_source_index")

cols = ["Damaged_Ehor",
        "Masonry_w", "Masonry_Ehor", "Masonry_FtmHor", "Masonry_FmHor"]

df_filtered, kept_idx, removed_idx, clusters = normalized_distance_filter(
    df,
    cols=cols, 
    scale="minmax", #zscore or minmax
    eps=0.000001,
    keep="first"
)

print(f"Kept {len(kept_idx)} of {len(df)} rows; removed {len(removed_idx)} near-duplicates.")

# Use kept indices to filter the original JSON list

source_indices = df.loc[kept_idx, "_source_index"].unique().tolist()
filtered_data = [data[i] for i in source_indices]

In [ ]:
with open("scenarios_results.json", "w") as f:
    json.dump(filtered_data, f, indent=4)